# GRO620 - Activité procédurale 1

Dans cette activité, nous allons principalement travailler sur les éléments nécessaires pour capter une image numériquement, les transformations entre repères 2D et 3D, et l'encodage numérique de la couleur.

In [1]:
# Préambule

import numpy as np
import cv2

import matplotlib.pyplot as plt
%matplotlib inline

## Si vous utilisez Google Colab, vous devez d'abord monter votre Google Drive
## où se trouve vos données. 
## Commentez les trois lignes suivantes en ajustant le chemin vers votre propre
## dossier :

# from google.colab import drive
# drive.mount('/content/gdrive')
# %cd /content/gdrive/MyDrive/gro620-e21

## Pour retrouver le chemin depuis Jupyter, vous pouvez utiliser ceci :
# !ls /content/gdrive/MyDrive


## Acquisition et caractéristiques de la lumière

### Q1.1

À partir de la figure 2.23 du livre de référence, décrivez en une phrase le rôle de chacune des étapes de la chaîne d'acquisition d'images numériques.

*(réponse ici)*\
__Optics__ : Lentille qui perment de diriger la lumière sur le senseur. Niveau de zoom Focus.<br>
__Aperture__ : Ouverture laissant passer plus ou moins de lumière.<br>
__Shutter__ : Temps d'acquisition de lumière du senseur.<br>
__Senseur__ : Converti la lumière en signaux électrique comme du voltage (CCD & CMOS)<br>
__Gain (ISO)__ : Sensibilité à la lumière (amplifie le voltage) peut amplifier le bruit<br>
__A/D__ : Analog to Digital convertisseur qui applique un gain électronique<br>
__Demosaic__ : Transform from a color filter array (CFA) into a full-color RGB image<br>
__Sharpen__ : Retirer du bruit permettant d'améliorer la qualité de l'image. Élimigne le flou dans l'image<br>
__White Balance__ : Ajuste la luminosité de l'image en fonction du point le plus blanc dans l'image. Ajuste les couleurs dans l'image<br>
__Gamma/curve__ : Donne une meilleure résolution à certaines valeurs du capteur (sombre ou claire) selon une courbe au lieu d'être linéaire<br>
__Compress__ : Réduire la taille du fichier de l'image ce qui peut induire des pertes<br>

### Q1.2

Quelle est la différence entre paramètres intrinsèques et extrinsèques du caméra ? Décrivez chaque type en une phrase.

*(réponse ici)*<br>
__Intrinsèques__ : Paramètres du capteur qui sont fixés par le fabricant du capteur propre à la caméra
- Focus : Niveau de zoom
- Distance focale
- Résolution
- Taille du capteur

__Extrinsèques__ : Paramètres du capteur qui sont modifiés par l'utilisateur (externe)
- Position de la caméra dans l'environnement (matrice de transformation)
- 

### Q1.3

Soit la configuration intrinsèque d'une caméra représentée par la matrice $K$ :

$$
K = \begin{bmatrix} 
 620 &   0 & 1024 \\ 
   0 & 620 &  512 \\ 
   0 &   0 &    1 
\end{bmatrix}
$$

$$
K = \begin{bmatrix} 
 f &   0 & cx \\ 
   0 & f &  cy \\ 
   0 &   0 &    1 
\end{bmatrix}
$$

Coordonnées du centre du capteur (cx, cy)

# Voir p.52

Le capteur de cette caméra a une taille de 30 mm x 15 mm.

Pouvez-vous estimer la distance focale en mm de la lentille de cette caméra à partir de la matrice $K$ ?

In [2]:
# Réponse ici.
K = np.array([[620.,   0., 1024.],
              [  0., 620.,  512.],
              [  0.,   0.,    1.]
])
f = 620 # pixels
# 1024 pixels = 15mm
# 620 = ?
f_mm = f*15/1024
print(str(f_mm) + " mm")

9.08203125 mm


### Q1.4

Dans le cadre de cet APP, nous considérons les caméras comme étant idéales, c'est-à-dire qu'on peut obtenir leurs caractéristiques intrinsèques et extrinsèques à partir de quelques paramètres seulement.

**a)** Qu'est-ce qui rend les vraies caméras non-idéales ? Nommez des facteurs autant pour les caractéristiques intrinsèques que extrinsèques.

*(réponse ici)*<br>
- Imperfection de la lentille (Abérration sphérique)
- Imperfection d'assemblage
- Coma (ressemble à la trainée d'une commète)
- Astigmatism
- Field curvature
- Distortion
- Chromatic aberration (dispersion de la lumière blanche)

**b)** Que doit on faire pour obtenir les caractéristiques d'une caméra non-idéale ?

*(réponse ici)*<br>
- Calibration de la caméra

### Q1.5

Dans cette image synthétique : 

![](https://upload.wikimedia.org/wikipedia/commons/c/cd/Specular_highlight.jpg)

(source: [Wikimedia Commons](https://commons.wikimedia.org/wiki/File:Specular_highlight.jpg))

**a)** Quelle(s) partie(s) correspondent à l'illumination diffuse et les reflets spéculaires ?

*(réponse ici)*<br>
__Illumination diffuse__ : La lumière se disperse dans tous les sens (surface matte, il n'y a pas de réflection)
- La couleur pleine des balles

__Reflets spéculaires__ : Réflection de la lumière (glossy) dépend de l'éclairage
- La partie qui est blanche

**b)** Quelle information est nécessaire pour déterminer les caractéristiques et emplacements exacts des sources de lumières dans cette image ? Vous pouvez répondre en utilisant des éléments de la *Bidirectional Reflectance Distribution Function* (BRDF).

*(réponse ici)*<br>
__Nécessite__ :
- La position des sphères
- La position de la caméra

On a pas toutes les infos nécessaires

### Q1.6

**a)** Pourquoi deux appareils de capture peuvent produire des valeurs RGB différentes d'une même couleur ? 

*(réponse ici)*<br>
- La calibration peut être différente
- Les paramètres pour le white balance etc sont différents

**b)** Que peut-on faire pour comparer numériquement des couleurs provenant de deux capteurs différents ?

*(réponse ici)*<br>
Utiliser l'espace de couleur XYZ (repère global) et faire des changement de repère de couleur pour les mettre les deux dans cet espace de couleur

## Repères et coordonnées

### Q2.1

Supposons ces 2 repères :

![](images_doc/proc1-q2_1-frames.png)

**a)** Trouvez la matrice homogène permettant de transformer un point du repère $\{1\}$ au repère $\{0\}$.

In [3]:
R1 = np.array([[1, 0, 0], [0, -1, 0], [0, 0, -1]]) # Rotation de 180° sur l'axe x
R2 = np.array([[0, 1, 0], [-1, 0, 0], [0, 0, 1]]) # Rotation de 90° sur l'axe y
R = R1 @ R2 # Rotation de 180° sur l'axe x puis de 90° sur l'axe y

Translation = np.array([[240], [80], [120], [1]])

T_intermediaire = np.vstack((R, np.zeros((1,3))))

T_10 = np.hstack((T_intermediaire, Translation))

print("Transform:\n", T_10)

Transform:
 [[  0.   1.   0. 240.]
 [  1.   0.   0.  80.]
 [  0.   0.  -1. 120.]
 [  0.   0.   0.   1.]]


**b)** Trouvez maintenant la transformation inverse.

In [4]:
T_01 = np.linalg.inv(T_10)
print("Inverse Transform:\n", T_01)

Inverse Transform:
 [[   0.    1.    0.  -80.]
 [   1.    0.    0. -240.]
 [  -0.   -0.   -1.  120.]
 [   0.    0.    0.    1.]]


**c)** Soit le point $p_0 = [8, 5, 1]^T$, un point dans le repère $\{0\}$. Trouvez $p_1$, ses coordonnées dans le repère $\{1\}$.

In [5]:
p_0 = [8, 5, 1, 1]
p_1 = [0, 0, 0, 0]


p_1 = T_01 @ p_0

print("Point dans repère 1:\n", p_1[0:3])

Point dans repère 1:
 [ -75. -232.  119.]


### Q2.2

Supposons maintenant que le repère $\{1\}$ représente une caméra avec les caractéristiques intrinsèques $K$ de la question Q1.3.

**a)** Trouvez la matrice de projection P complète permettant de projeter un point $p$ décrit dans le repère $\{0\}$.

# Voir page 54 Livre
$P  = K [R | T]$ _où R | T représente la matrice de rotation et le vecteur de transformation k(3x3) * R|t(3x4) = (3x4)_

$\tilde{P} = \tilde{K} E$ _Ici c'est juste de rajouter la ligne (1x4) de [0, 0, 0, 1] à K pour donner ktilde et que RT soit ta matrice de transformation_

$x_s \sim \tilde{P} \bar{p_w}$

$x_s = (X_s, Y_s, 1, d) $

$d = 1 / z$

Dans notre cas, $E$ correspond à $T_{01}$, car on souhaite passer du repère global à celui de la caméra, $p_w$ est donc $p_0$ et $x_s$ continendra les coordonnées normalisées dans l'image.

$d$ correspond à la disparité, ou $1/z$, où $z$ et la distance du point dans le repère de la caméra (donc en Z, où Z augmente avec la distance).

In [10]:
#print(K) # Si vous n'avez pas réutilisé la variable K, elle aura toujours la même valeur qu'à la question Q1.3.

k_tilde = np.hstack( ( np.vstack( (K, np.zeros((1,3))) ), np.array([[0], [0], [0], [1]]) ) )

p_tilde = k_tilde @ T_01 # 4x4

print("P_tilde:\n", p_tilde)



P_tilde:
 [[ 0.000e+00  6.200e+02 -1.024e+03  7.328e+04]
 [ 6.200e+02  0.000e+00 -5.120e+02 -8.736e+04]
 [ 0.000e+00  0.000e+00 -1.000e+00  1.200e+02]
 [ 0.000e+00  0.000e+00  0.000e+00  1.000e+00]]


**b)** Soit le point $p_0 = [0.250, 0.010, 0.000]$ en m. Trouvez le point $x_s$, les coordonnées du point $p_0$ perçu par la caméra.

In [22]:
p_0 = np.array([250, 10, 0.000, 1])
x_s = p_tilde @ p_0

print(x_s)

x_s_normalized = x_s / x_s[2]

print(x_s_normalized)

[7.948e+04 6.764e+04 1.200e+02 1.000e+00]
[6.62333333e+02 5.63666667e+02 1.00000000e+00 8.33333333e-03]


## Reprojection 2D à 3D

### Q3.1

Supposons que le plan XY du repère $\{0\}$ est un convoyeur. Quelle serait sa largeur maximale (mesurée sur l'axe Y) si on souhaite que la caméra la capte au complet dans son image ? 

In [8]:
l_conv = 0

### Q3.2

Soit le point $x_s = [120, 200]$, un point dans l'image perçu par la caméra décrite plus haut. On suppose que le point perçu se trouve sur le plan XY du repère $\{0\}$. Trouvez les coordonnées du point $p_0$ qui correspond à ce même point dans le repère $\{0\}$.

In [46]:
x_s2 = np.array([120, 200, 1, 1/120]) * 120

print(x_s2)

p_inv = np.linalg.inv(p_tilde)

p_w = p_inv @ x_s2

print(p_w)

[1.44e+04 2.40e+04 1.20e+02 1.00e+00]
[179.61290323 -94.96774194   0.           1.        ]
